In [116]:
! pip install -U polygon-api-client



[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from polygon import RESTClient
import time
import pandas as pd
import os

In [6]:
api_key = os.getenv("POLYGON_KEY")


In [7]:
client = RESTClient(api_key=api_key)


In [17]:
help(client.list_aggs)

Help on method list_aggs in module polygon.rest.aggs:

list_aggs(ticker: str, multiplier: int, timespan: str, from_: Union[str, int, datetime.datetime, datetime.date], to: Union[str, int, datetime.datetime, datetime.date], adjusted: Optional[bool] = None, sort: Union[str, polygon.rest.models.common.Sort, NoneType] = None, limit: Optional[int] = None, params: Optional[Dict[str, Any]] = None, raw: bool = False, options: Optional[polygon.rest.models.request.RequestOptionBuilder] = None) -> Union[Iterator[polygon.rest.models.aggs.Agg], urllib3.response.HTTPResponse] method of polygon.rest.RESTClient instance
    List aggregate bars for a ticker over a given date range in custom time window sizes.
    
    :param ticker: The ticker symbol.
    :param multiplier: The size of the timespan multiplier.
    :param timespan: The size of the time window.
    :param from_: The start of the aggregate time window as YYYY-MM-DD, a date, Unix MS Timestamp, or a datetime.
    :param to: The end of the a

In [4]:
import pandas as pd
MAX_BAR_LIMIT = 50_000
df = pd.DataFrame(client.list_aggs(ticker='O:AAPL240802C00170000',multiplier=5,timespan='minute',from_='2024-03-01',to='2025-01-01',limit=MAX_BAR_LIMIT))
df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,40.90,40.90,40.90,40.90,6,40.90,1719327000000,2,None
1,44.90,44.90,44.90,44.90,1,44.90,1719497700000,1,None
2,49.88,49.88,49.88,49.88,4,49.88,1719929400000,1,None
3,51.06,51.06,51.06,51.06,1,51.06,1719933000000,1,None
4,50.12,50.12,50.12,50.12,6,50.12,1719938100000,1,None


### Constrats





In [5]:
help(client.list_options_contracts)

Help on method list_options_contracts in module polygon.rest.reference:

list_options_contracts(underlying_ticker: Optional[str] = None, underlying_ticker_lt: Optional[str] = None, underlying_ticker_lte: Optional[str] = None, underlying_ticker_gt: Optional[str] = None, underlying_ticker_gte: Optional[str] = None, contract_type: Optional[str] = None, expiration_date: Union[str, datetime.date, NoneType] = None, expiration_date_lt: Union[str, datetime.date, NoneType] = None, expiration_date_lte: Union[str, datetime.date, NoneType] = None, expiration_date_gt: Union[str, datetime.date, NoneType] = None, expiration_date_gte: Union[str, datetime.date, NoneType] = None, as_of: Union[str, datetime.date, NoneType] = None, strike_price: Optional[float] = None, strike_price_lt: Optional[float] = None, strike_price_lte: Optional[float] = None, strike_price_gt: Optional[float] = None, strike_price_gte: Optional[float] = None, expired: Optional[bool] = None, limit: Optional[int] = None, sort: Union[s

####  Stock

In [62]:
underlying_stock = 'FMC'
# BHP
# FMC

##### Start Date

In [63]:
start_date = '2025-01-23'

### Contrats actifs

In [60]:
active_contracts = []
for option in client.list_options_contracts(underlying_ticker=underlying_stock,expiration_date_gte=start_date,limit=1000):
  active_contracts.append(option)
  # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
  if len(active_contracts) % 5000 == 0:
     time.sleep(60)

In [61]:
active_contracts[0:3]

[]

### Contrats expirés

In [19]:
expired_contracts = []
for option in client.list_options_contracts(underlying_ticker=underlying_stock,expiration_date_gte=start_date,limit=1000,expired=True):
  expired_contracts.append(option)
  # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
  if len(active_contracts) % 5000 == 0:
     time.sleep(60)

In [20]:
expired_contracts[0:3]

[]

In [21]:
set([contract.ticker for contract in active_contracts]).intersection(set([contract.ticker for contract in expired_contracts]))

set()

In [22]:
len(active_contracts),len(expired_contracts)

(334, 0)

### Tous les contrats

In [23]:
options_contracts = expired_contracts + active_contracts

In [24]:
pd.DataFrame(options_contracts).assign(NEW_COL='MY NEW COL!')

,additional_underlyings,cfi,contract_type,correction,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker,NEW_COL
0,None,OCASPS,call,None,american,2025-02-21,BATO,100,27.5,O:BHP250221C00027500,BHP,MY NEW COL!
1,None,OCASPS,call,None,american,2025-02-21,BATO,100,30.0,O:BHP250221C00030000,BHP,MY NEW COL!
2,None,OCASPS,call,None,american,2025-02-21,BATO,100,32.5,O:BHP250221C00032500,BHP,MY NEW COL!
3,None,OCASPS,call,None,american,2025-02-21,BATO,100,35.0,O:BHP250221C00035000,BHP,MY NEW COL!
4,None,OCASPS,call,None,american,2025-02-21,BATO,100,37.5,O:BHP250221C00037500,BHP,MY NEW COL!
...,...,...,...,...,...,...,...,...,...,...,...,...
329,None,OPASPS,put,None,american,2027-01-15,BATO,100,70.0,O:BHP270115P00070000,BHP,MY NEW COL!
330,None,OPASPS,put,None,american,2027-01-15,BATO,100,75.0,O:BHP270115P00075000,BHP,MY NEW COL!
331,None,OPASPS,put,None,american,2027-01-15,BATO,100,80.0,O:BHP270115P00080000,BHP,MY NEW COL!
332,None,OPASPS,put,None,american,2027-01-15,BATO,100,85.0,O:BHP270115P00085000,BHP,MY NEW COL!


## calcul putcall ratio

In [ ]:
import time
df_list = []
for i,contract in enumerate(options_contracts):
    df_list.append(pd.DataFrame(client.list_aggs(ticker=contract.ticker,multiplier=1,timespan='day',from_=start_date,to='2025-01-24',limit=5000)).assign(ticker=contract.ticker))
    if i%100==0:
      print(f"Getting data for contract {i} of {len(options_contracts)}")
  # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
    if len(df_list) % 5 == 0:
        time.sleep(60)

  # Combine the list into of dataframes into 1 dataframe and clean up the timestamp format
df = pd.concat(df_list)
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')

  # Merge with the contracts to get the contract types
df=pd.DataFrame(options_contracts).merge(df,on='ticker')
df['call_volume'] = np.where(df['contract_type'] == 'call',df['volume'],0)
df['put_volume'] = np.where(df['contract_type'] == 'put',df['volume'],0)
df_grouped = df.groupby('timestamp')[['put_volume','call_volume','volume']].sum()
df_grouped['put_call_ratio'] = df_grouped['put_volume']/df_grouped['call_volume']
df_grouped


In [48]:
df_grouped

,put_volume,call_volume,volume,put_call_ratio
timestamp,,,,
2025-01-23 05:00:00,1528.0,996.0,2524.0,1.534137


In [ ]:

  # Calculate the Daily Put-Call Volume Ratio
   df['call_volume'] = np.where(df['contract_type'] == 'call',df['volume'],0)
   df['put_volume'] = np.where(df['contract_type'] == 'put',df['volume'],0)
   df_grouped = df.groupby('timestamp')[['put_volume','call_volume','volume']].sum()
   df_grouped['put_call_ratio'] = df_grouped['put_volume']/df_grouped['call_volume']


In [49]:
df_grouped

,put_volume,call_volume,volume,put_call_ratio
timestamp,,,,
2025-01-23 05:00:00,1528.0,996.0,2524.0,1.534137


### Avoir les données journalières

In [28]:
df_list = []
for contract in options_contracts:
  df_list.append(pd.DataFrame(client.list_aggs(ticker=contract.ticker,multiplier=1,timespan='day',from_=start_date,to='2025-01-24',limit=50000)).assign(ticker=contract.ticker))
  # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
  if len(df_list) % 5 == 0:
     time.sleep(60)

In [65]:
df_list[0].head()

,ticker


In [66]:
len(df_list)

334

##### Concaténer

In [31]:
df = pd.concat(df_list)

In [32]:
df.columns

Index(['ticker', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'timestamp',
       'transactions', 'otc'],
      dtype='object')

In [33]:
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')

#### avoir contrat type

In [34]:
pd.DataFrame(options_contracts)

,additional_underlyings,cfi,contract_type,correction,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker
0,None,OCASPS,call,None,american,2025-02-21,BATO,100,27.5,O:BHP250221C00027500,BHP
1,None,OCASPS,call,None,american,2025-02-21,BATO,100,30.0,O:BHP250221C00030000,BHP
2,None,OCASPS,call,None,american,2025-02-21,BATO,100,32.5,O:BHP250221C00032500,BHP
3,None,OCASPS,call,None,american,2025-02-21,BATO,100,35.0,O:BHP250221C00035000,BHP
4,None,OCASPS,call,None,american,2025-02-21,BATO,100,37.5,O:BHP250221C00037500,BHP
...,...,...,...,...,...,...,...,...,...,...,...
329,None,OPASPS,put,None,american,2027-01-15,BATO,100,70.0,O:BHP270115P00070000,BHP
330,None,OPASPS,put,None,american,2027-01-15,BATO,100,75.0,O:BHP270115P00075000,BHP
331,None,OPASPS,put,None,american,2027-01-15,BATO,100,80.0,O:BHP270115P00080000,BHP
332,None,OPASPS,put,None,american,2027-01-15,BATO,100,85.0,O:BHP270115P00085000,BHP


In [35]:
df=pd.DataFrame(options_contracts).merge(df,on='ticker')

In [36]:
df[['timestamp','contract_type','volume']]

,timestamp,contract_type,volume
0,2025-01-23 05:00:00,call,19.0
1,2025-01-23 05:00:00,call,37.0
2,2025-01-23 05:00:00,call,53.0
3,2025-01-23 05:00:00,call,79.0
4,2025-01-23 05:00:00,call,19.0
...,...,...,...
70,2025-01-23 05:00:00,call,1.0
71,2025-01-23 05:00:00,call,5.0
72,2025-01-23 05:00:00,call,1.0
73,2025-01-23 05:00:00,put,10.0


####  Put-Call Volume Ratio





In [37]:
import numpy as np
df['call_volume'] = np.where(df['contract_type'] == 'call',df['volume'],0)
df['put_volume'] = np.where(df['contract_type'] == 'put',df['volume'],0)
df_grouped = df.groupby('timestamp')[['put_volume','call_volume','volume']].sum()
df_grouped['put_call_ratio'] = df_grouped['put_volume']/df_grouped['call_volume']
df_grouped

,put_volume,call_volume,volume,put_call_ratio
timestamp,,,,
2025-01-23 05:00:00,1528.0,996.0,2524.0,1.534137


In [70]:
df_grouped.to_csv("../../new_data/daily_put_call/daily_put_call_BHP.csv")

#### Plot put call ratio

In [44]:
import plotly.express as px
fig = px.line(df_grouped, y='put_call_ratio', height=720, width=1280)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'timestamp=%{x}<br>put_call_ratio=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2025, 1, 23, 5, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([1.53413655]),
              'yaxis': 'y'}],
    'layout': {'height': 720,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'width': 1280,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timestamp'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'put_call_ratio'}}}
})

In [39]:
import time
import numpy as np
import pandas as pd
from polygon import RESTClient


def get_put_call_ratio(underlying_stock,start_date='2023-01-23'):

  active_contracts = []
  for option in client.list_options_contracts(underlying_ticker=underlying_stock,expiration_date_gte=start_date,limit=1000):
    active_contracts.append(option)
    # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
    if len(active_contracts) % 5000 == 0:
       time.sleep(60)
  expired_contracts = []
  for option in client.list_options_contracts(underlying_ticker=underlying_stock,expiration_date_gte=start_date,limit=1000,expired=True):
    expired_contracts.append(option)
    # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
    if len(active_contracts) % 5000 == 0:
       time.sleep(60)

  # Combine the expired and active contracts
  options_contracts = expired_contracts + active_contracts

  print(f"Total Contracts: {len(options_contracts)}")
  # Get bar data for all the option contracts defined above
  df_list = []
  for i,contract in enumerate(options_contracts):
    df_list.append(pd.DataFrame(client.list_aggs(ticker=contract.ticker,multiplier=1,timespan='week',from_=start_date,to='2025-01-01',limit=50000)).assign(ticker=contract.ticker))
    if i%100==0:
      print(f"Getting data for contract {i} of {len(options_contracts)}")
  # Uncomment both lines belows if you hare using the "Free Tier" since there is max of 5 API calls per minute
  if len(df_list) % 5 == 0:
     time.sleep(60)

  # Combine the list into of dataframes into 1 dataframe and clean up the timestamp format
  df = pd.concat(df_list)
  df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')

  # Merge with the contracts to get the contract types
  df=pd.DataFrame(options_contracts).merge(df,on='ticker')

  # Calculate the Daily Put-Call Volume Ratio
  df['call_volume'] = np.where(df['contract_type'] == 'call',df['volume'],0)
  df['put_volume'] = np.where(df['contract_type'] == 'put',df['volume'],0)
  df_grouped = df.groupby('timestamp')[['put_volume','call_volume','volume']].sum()
  df_grouped['put_call_ratio'] = df_grouped['put_volume']/df_grouped['call_volume']
  return df_grouped

In [ ]:
FMC = get_put_call_ratio("FMC",start_date="2025-01-20")

In [ ]:
BHP = get_put_call_ratio("BHP",start_date='2025-01-20')

In [81]:
df_put_call_nvda = get_put_call_ratio('AAPL')
df_put_call_tsla = get_put_call_ratio('NVDA')


MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v3/reference/options/contracts?underlying_ticker=AAPL&expiration_date.gte=2024-01-01&limit=1000 (Caused by ResponseError('too many 429 error responses'))

### Combine the two

In [45]:
df_pc_comparison=pd.concat([FMC,BHP])

NameError: name 'FMC' is not defined

### Make a Plotly Plot of The Two

In [50]:
import plotly.express as px

# Assuming df_pc_comparison is your DataFrame with 'put_call_ratio' and 'ticker' columns
fig = px.line(df_pc_comparison, y='put_call_ratio', color='ticker', height=720, width=1280)

# Update y-axis to percentage format and improve overall appearance
fig.update_yaxes(tickformat='.1%', title='Put-Call Volume Ratio (%)')
fig.update_xaxes(title='Date')

# Update layout for a nicer format
fig.update_layout(
    title={
        'text': 'Put-Call Volume Ratio Over Time',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 40}  # Increase the title font size
    },
    xaxis_title='Date',
    yaxis_title='Put-Call Volume Ratio (%)',
    legend_title='Ticker',
    font=dict(family="Arial", size=30, color="RebeccaPurple"),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

# Optionally, add grid lines for better readability
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Increase the line thickness
fig.update_traces(line=dict(width=14))  # Adjust the width as desired

fig.show()


NameError: name 'df_pc_comparison' is not defined